# Data Handler
Soubour určený k manipulaci s csv soubory, neboli příslušnými lékařskými zprávami

In [ ]:
import pandas as pd
from scripts import config
import os
import markdown
#import re

base_root = config.CSV_DATA_ROOT
files = os.listdir(base_root)
crohn = os.path.join(base_root, files[0])
crohn_2 = os.path.join(base_root, files[1])
stroke = os.path.join(base_root, files[2])

### Spočítá počet znaků každého nadpisu a zprůměruje je

In [ ]:
df: pd.DataFrame = pd.read_csv(stroke)

# 1️⃣ Získáme "čistý název" z title (odstraníme prefixy jako "041/000 - " apod.)
df["clean_title"] = df["title"].str.extract(r"-\s*(.*)$")[0].str.strip()
# 2️⃣ Počet slov v textu
df["word_count"] = df["text"].fillna("").str.split().str.len()

# 3️⃣ Filtrovat jen ty řádky, které mají clean_title v TITLES_CROHN
filtered = df[df["clean_title"].isin(config.TITLES_STROKE)]

# 4️⃣ Spočítat průměrný / celkový počet slov pro každý title
summary = (
    filtered.groupby("clean_title")["word_count"]
    .mean()  # nebo .sum(), podle potřeby
    .reset_index()
    .sort_values(by="word_count", ascending=False)
)

with open("summary_c1.txt", 'w', encoding='utf-8') as f:
    f.write(summary.to_string())
    f.write("\n")

### Script k nalezení "title"
Funkce má za úkol odstranit čísla a znaky ze sloupce 'title' a ponechat jen skutečný název vyšetření

In [ ]:
texts = []

for _, row in df.iterrows():
    print(str(row['title']))# každá řádka = Series s hodnotami
    if config.TITLES_CROHN[2] in str(row['title']):  # přístup ke sloupci 'title'
        texts.append(str(row['text']))  # přístup ke sloupci 'text'
        
with open("output.txt", 'w', encoding='utf-8') as f:
    for text in texts:
        f.write(text)
        f.write("\n")
        f.write("---------------------------------------------------------------------")
        f.write("\n")

### Testování správných slov
Načtu jednotlivá slova ze zadání a z výstupů a zjistím kolik slov si model vymyslel.

In [ ]:
text_path = config.REPORTS_ROOT
result_path = config.RESULT_ROOT

# 1️⃣ Načtení slov z medical reportu
words = []
text_file = os.path.join(text_path, "01_crohn.txt")

with open(text_file, "r", encoding="utf-8") as fr:
    for line in fr:
        words.extend(line.strip().split())

# Vytvoř množinu pro rychlé porovnání (malá písmena)
valid_words = set(word.lower().strip(",.()[];:!?") for word in words if word.strip())

print(f"✅ Načteno {len(valid_words)} unikátních slov z {text_file}\n")


# 2️⃣ Načtení markdown souborů z result_path
files = [f for f in os.listdir(result_path) if os.path.isfile(os.path.join(result_path, f))]
markdown_html_results = {}

for file in files:
    if file.endswith(".md") and "r01" in file:
        path = os.path.join(result_path, file)
        with open(path, "r", encoding="utf-8") as f:
            md_text = f.read()
            html_content = markdown.markdown(md_text)
            markdown_html_results[file] = html_content

print(f"✅ Načteno {len(markdown_html_results)} markdown souborů\n")


# 3️⃣ Porovnání slov z md souborů se slovní zásobou reportu
unknown_words_total = set()
unknown_words_llm = {}


for filename, html_content in markdown_html_results.items():
    # Odstraň HTML tagy a rozbij text na slova
    text_only = re.sub(r"<[^>]+>", "", html_content)
    md_words = re.findall(r"\b\w+\b", text_only.lower())

    unknown_words = set(w for w in md_words if w not in valid_words)
    unknown_words_total.update(unknown_words)
    unknown_words_llm[filename] = unknown_words
    
    print(f"📄 {filename}: {len(unknown_words)} neznámých slov")

print("==============================================")
print(f"🔍 Celkem nalezeno {len(unknown_words_total)} unikátních 'vymyšlených' slov.")
print("==============================================")

output_path = "unknown_words.txt"
with open(output_path, 'w', encoding='utf-8') as wr:
    for key, values in unknown_words_llm.items():
        wr.write(f"------------ {key} ------------\n")
        for _ in range(len(unknown_words_llm[key])):
            wr.write(unknown_words_llm[key].pop())
            wr.write("\n")
        wr.write("\n")

AttributeError: module 'scripts.config' has no attribute 'REPORTS_ROOT'